# Machine Learning - Classification: Predict

#### Team members : Ntokozo Hlatshwayo, Sizwe Khumalo

In [22]:

#importing libraries
import numpy as np
import pandas as pd
%matplotlib inline

# for all NLP related operations on text
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.classify import NaiveBayesClassifier
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import unidecode
import wordninja

nltk.download('stopwords')
nltk.download('wordnet')
stopwords = set(stopwords.words("english"))
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\petro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\petro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
import unidecode
import wordninja

In [4]:
#importing datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_with_no_labels.csv')

In [5]:
#datasets summary
display(train_df.head())
display(test_df.head())

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


The train datasets contains 3 columns of all have no null values.

## Data Preprocessing

In [14]:
def clean_tweet(text):
    
    # characters to lower-case
    text=text.lower()
    
    # remove twitter handles
    text= re.sub(r'@\S+', '',text) 
    
    # remove urls and RT
    text = re.sub(r'^rt ','', re.sub(r'https://t.co/\w+', '', text).strip())
      
    # replace unidecode characters
    text=unidecode.unidecode(text)
      
    # regex only keeps characters
    text= re.sub(r"[^a-zA-Z+']", ' ',text)
    
    # keep words with length>1 only
    text=re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ') 

    # split words like 'whatisthis' to 'what is this'
    def preprocess_wordninja(sentence):      
        def split_words(x):
            x=wordninja.split(x)
            x= [word for word in x if len(word)>1]
            return x
        new_sentence=[ ' '.join(split_words(word)) for word in sentence.split() ]
        return ' '.join(new_sentence)
    
    text=preprocess_wordninja(text)
    
    # regex removes repeated spaces, strip removes leading and trailing spaces
    text= re.sub("\s[\s]+", " ",text).strip()  
    
    return text

In [15]:
## Preprocessing Text
train_df['message']=train_df['message'].apply(lambda x: clean_tweet(x))
test_df['message']=test_df['message'].apply(lambda x: clean_tweet(x))

In [16]:
train_df.head()

,sentiment,message,tweetid
0,1,poly sci major epa chief doesn't think carbon ...,625221
1,1,it's not like we lack evidence of anthropogeni...,126103
2,2,researchers say we have three years to act on ...,698562
3,1,today in maker wired was pivotal year in the w...,573736
4,1,it's and racist sexist climate change denying ...,466954


In [18]:
clean_tweet('RT @thenation: Mike Pence doesn’t believe in global warming or that smoking causes lung cancer. https://t.co/gvWYaauU8R ')

"mike pence doesn't believe in global warming or that smoking causes lung cancer"